##**Code Section**##

- Colab Environment Setting

- Data Augmentation
 - file 구조(/dataset/{건물 방향 class label}/{integer}.jpg)
- Model
 - Pretrained model (데규모 데이터셋으로 학습된 특징 내포, 특징 추출의 일반화가능)
 - Resnet18 이용(작은 모델, 적은 데이터로도 overfitting 되지 않음)
- Train
- Test


<br>

**Training Section**

1. sub-class 없이 여러면의 사진을 전부 하나의 class로 학습시키고 test

2. sub-class만을 이용하여 학습 시키기

 - test 시에 sub-class 합을 이용
 - test 시에 각 sub-class를 이용

3. 각 건물의 sub-class 합을 통해 학습시키고 test

## **Colab Env Setting**

In [ ]:
# Mount Google Drive in /content/drive directory

from google.colab import drive
drive.mount('/content/drive')

# Change current directory

import os
os.chdir('/content/drive/MyDrive/Project')

Mounted at /content/drive


In [ ]:
##### DO NOT MODIFY #####
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import numpy
import cv2
from google.colab.patches import cv2_imshow
##### DO NOT MODIFY #####

In [ ]:
import torch

print(torch.cuda.is_available())

True


In [ ]:
!pip install pyheif
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 23.7 MB/s eta 0:00:00


## **Data Preprocessing and Augmentation**

In [ ]:
%%writefile building_info.py
building_label_to_name = {
    1: "정문안내소(정면)",
    2: "보노보노",
    3: "커뮤니티센터",
    4: "토르망치",
    5: "201동 본관",
    6: "101동",
    7: "103동",
    8: "105동",
    9: "107동",
    10: "109동",
    11: "114동 경영관",
    12: "203동 학관",
    13: "205동 스센",
    14: "다리 1(담배다리)",
    15: "다리 4(버정옆)",
    16: "다리 5(고래다리)",
    17: "다리 6(테니스장앞)",
    18: "대운",
    19: "바보계단",
    20: "이차전지 산학연 연구센터",
    21: "해수자원화기술 연구센터",
    22: "복합재료기술 연구센터",
    23: "산학협력관",
    24: "과일집",
    25: "거리변위다리",
    26: "식당동",
    27: "오렌지다리",
    28: "학술정보관"
}
class_to_building_label = {
    0: 1,
    1: 1,
    2: 1,
    3: 2,
    4: 3,
    5: 4,
    6: 4,
    7: 5,
    8: 5,
    9: 5,
    10: 5,
    11: 6,
    12: 6,
    13: 6,
    14: 6,
    15: 7,
    16: 7,
    17: 7,
    18: 7,
    19: 8,
    20: 8,
    21: 8,
    22: 8,
    23: 9,
    24: 9,
    25: 9,
    26: 10,
    27: 10,
    28: 10,
    29: 10,
    30: 11,	#114 경영관(정면)
    31: 11,	#114 경영관(우측)
    32: 11,	#114 경영관(뒷면)
    33: 11,	#114 경영관(좌측)
    34: 12,	#203 학관(정면)
    35: 12,	#203 학관(옆면)
    36: 12,	#203 학관(뒷면)
    37: 13,	#205 스센(정면)
    38: 13,	#205 스센(좌측)
    39: 13,	#205 스센(뒷면)
    40: 13,	#205 스센(우측)
    41: 14,	#담배다리(기숙사쪽)
    42: 14,	#담배다리(스센쪽)
    43: 15,	#본관옆 다리(본관쪽)
    44: 15,	#본관옆 다리(대운쪽)
    45: 16,	#고래다리(출구)
    46: 16,	#고래다리(입구)
    47: 17,	#테니스장다리(테니스장쪽)
    48: 17,	#테니스장다리(테니스장기준우측)
    49: 17,	#테니스장다리(스센쪽)
    50: 17,	#테니스장다리(테니스장기준 좌측)
    51: 18,	#대운(트랙나옴)
    52: 18,	#대운(트랙안나옴)
    53: 19,	#바보계단(경영관쪽)
    54: 19,	#바보계단(스센쪽)
    55: 19,	#바보계단(학관쪽)
    56: 20,		#111 이차전지 산학연 연구센터
    57: 20,		#111 이차전지 산학연 연구센터
    58: 20,		#111 이차전지 산학연 연구센터
    59: 20,		#111 이차전지 산학연 연구센터
    60: 21,		#113 해수자원화기술 연구센터
    61: 21,		#113 해수자원화기술 연구센터
    62: 21,		#113 해수자원화기술 연구센터
    63: 21,		#113 해수자원화기술 연구센터
    64: 22,		#123 복합재료기술 연구센터
    65: 22,		#123 복합재료기술 연구센터
    66: 22,		#123 복합재료기술 연구센터
    67: 22,		#123 복합재료기술 연구센터
    68: 23,		#251 산학협력관
    69: 23,		#251 산학협력관
    70: 23,		#251 산학협력관
    71: 23,		#251 산학협력관
    72: 24,		#125 과일집
    73: 24,		#125 과일집
    74: 24,		#125 과일집
    75: 24,		#125 과일집
    76: 25,		#거리변위다리
    77: 25,		#거리변위다리
    78: 26,		#206 식당동
    79: 26,		#206 식당동
    80: 26,		#206 식당동
    81: 26,		#206 식당동
    82: 27,		#오렌지다리
    83: 27,		#오렌지다리
    84: 28,		#202 학술정보관
    85: 28,		#202 학술정보관
    86: 28,		#202 학술정보관
    87: 28		#202 학술정보관
}

def get_building_name(label_num):
  return building_label_to_name.get(label_num, "Unknown")

def get_building_label(class_num):
  return class_to_building_label.get(class_num, "Unknown Building")

num_classes = 88
dataset_path = './dataset'

Overwriting building_info.py


In [ ]:
# Preprocessing, JPG, HEIC -> .jpg and 이미지 이름 0.jpg, 1.jpg ... 로 변경

import os
from pathlib import Path
from PIL import Image
import pyheif  # pyheif 라이브러리 추가


num_classes = 88
dataset_path = './dataset'



# 이미지를 .jpg로 변환하는 함수
def convert_image_to_jpg(image_path, output_path):
    try:
        # HEIC 파일인 경우 pyheif를 사용하여 변환
        if image_path.suffix.lower() == '.heic':
            heif_file = pyheif.read(image_path)  # HEIC 파일 읽기
            img = Image.frombytes(
                heif_file.mode,
                heif_file.size,
                heif_file.data,
                "raw",
                heif_file.mode,
                heif_file.stride
            )
        else:
            img = Image.open(image_path)  # HEIC 외의 파일은 PIL로 열기

        img.convert('RGB').save(output_path, 'JPEG')  # .jpg로 저장
        print(f"변환 완료: {image_path} -> {output_path}")
    except Exception as e:
        print(f"이미지 변환 중 오류 발생: {image_path}, {e}")

# 폴더 내 이미지 파일 이름 변경 및 포맷 변환
def rename_images_in_folder(folder_path):
    folder = Path(folder_path)

    if not folder.exists() or not folder.is_dir():
        print(f"폴더가 존재하지 않거나 유효하지 않습니다: {folder_path}")
        return

    image_files = [f for f in folder.iterdir() if f.is_file()]

    for idx, image_file in enumerate(image_files):
        # 새로운 파일명 생성 (0.jpg, 1.jpg, ...)
        new_name = f"{idx}.jpg"
        new_path = folder / new_name

        # 이미지 파일을 .jpg로 변환 후 저장
        convert_image_to_jpg(image_file, new_path)

    print("이름 변경 및 포맷 변환 완료.")

# 예시로 dataset_path와 num_classes가 이미 정의되어 있다고 가정
for i in range(num_classes):
    path = f'{dataset_path}/{str(i)}'
    print(path)
    rename_images_in_folder(path)


./dataset/0
변환 완료: dataset/0/IMG_8404.JPG -> dataset/0/0.jpg
변환 완료: dataset/0/IMG_8405.JPG -> dataset/0/1.jpg
변환 완료: dataset/0/IMG_8402.JPG -> dataset/0/2.jpg
변환 완료: dataset/0/IMG_8403.JPG -> dataset/0/3.jpg
변환 완료: dataset/0/IMG_8399.JPG -> dataset/0/4.jpg
변환 완료: dataset/0/IMG_8401.JPG -> dataset/0/5.jpg
변환 완료: dataset/0/IMG_8400.JPG -> dataset/0/6.jpg
이름 변경 및 포맷 변환 완료.
./dataset/1
변환 완료: dataset/1/IMG_8421.JPG -> dataset/1/0.jpg
변환 완료: dataset/1/IMG_8415.JPG -> dataset/1/1.jpg
변환 완료: dataset/1/IMG_8422.JPG -> dataset/1/2.jpg
변환 완료: dataset/1/IMG_8406.JPG -> dataset/1/3.jpg
변환 완료: dataset/1/IMG_8418.JPG -> dataset/1/4.jpg
변환 완료: dataset/1/IMG_8412.JPG -> dataset/1/5.jpg
변환 완료: dataset/1/IMG_8414.JPG -> dataset/1/6.jpg
변환 완료: dataset/1/IMG_8411.JPG -> dataset/1/7.jpg
변환 완료: dataset/1/IMG_8407.JPG -> dataset/1/8.jpg
변환 완료: dataset/1/IMG_8416.JPG -> dataset/1/9.jpg
변환 완료: dataset/1/IMG_8408.JPG -> dataset/1/10.jpg
변환 완료: dataset/1/IMG_8410.JPG -> dataset/1/11.jpg
변환 완료: dataset/1/IMG_8420

**10시간 고민했던 문제 !!!**


<br>
DataLoader에서는 dataset 내부 폴더들의 이름을 y값으로 지정함.

하지만 엄밀히 따지자면 이름들을 idx로 바꿔서 저장.


이때, 이름들을 str로 취급.

그래서 classification을 할 경우, bird, apple, lion 3개의 폴더에 각각의 이미지가 있을 때, model의 output값에서 0번째 index는 apple이 될 것. 사전식 정렬을 해서 class(이름들) to idx(index값)로 변경해서, 저장하기 때문임.

나는 0부터 87까지의 이름을 갖는 총 88개의 폴더를 생성했지만, 여기서 모델이 예측을 하면 꼭 일관적으로(같은 클래스의 이미지들은 다 같은 클래스를 예측) 틀린 클래스를 예측했다. 하지만 training과 test 결과는 좋았다. 꼭 개별 이미지를 사용해서 예측한 결과만 이상했다.

이유가 폴더 명이 str형식이어서, 0, 1, 2, 3, 4, 순서로 idx도 동일하게 생성된 것이 아니라 0, 1, 10, 11, 형식으로 취급해서 이것들이 0, 1, 2, 3, 4로 저장되었기 때문이다.

이것을 확인하는 class to idx가 다음고 같다.



In [ ]:
from torchvision import datasets

# 데이터셋 로드
dataset = datasets.ImageFolder(root='./dataset')

# 폴더 이름과 레이블 번호 매핑 확인
print(dataset.class_to_idx)


# 매핑을 뒤집기
idx_to_class = {v: int(k) for k, v in class_to_idx.items()}

# 예측된 값 -> 원래 클래스 이름으로 변환
def get_original_class(predicted_label):
    """
    예측된 숫자를 원래 폴더 이름으로 변환
    :param predicted_label: 모델이 예측한 레이블 (int)
    :return: 원래 폴더 이름 (str)
    """
    return idx_to_class.get(predicted_label, "Unknown")  # 존재하지 않는 레이블이면 "Unknown" 반환


{'0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11, '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22, '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33, '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44, '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '58': 54, '59': 55, '6': 56, '60': 57, '61': 58, '62': 59, '63': 60, '64': 61, '65': 62, '66': 63, '67': 64, '68': 65, '69': 66, '7': 67, '70': 68, '71': 69, '72': 70, '73': 71, '74': 72, '75': 73, '76': 74, '77': 75, '78': 76, '79': 77, '8': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '9': 87}


In [ ]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets.folder import make_dataset, default_loader

# 숫자 순서로 폴더를 정렬하도록 커스텀 Dataset 생성
class NumericFolderDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, target_transform=None, loader=default_loader):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

        # 폴더 이름을 숫자 순으로 정렬
        self.classes, self.class_to_idx = self._find_classes_numeric_order(root)
        self.samples = make_dataset(root, self.class_to_idx, extensions=None, is_valid_file=None)

    def _find_classes_numeric_order(self, directory):
        classes = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))], key=int)
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

    def __len__(self):
        return len(self.samples)

# Augmentation 설정
torch.manual_seed(42)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지를 224x224로 크기 변경 (비율 유지)
    transforms.RandomHorizontalFlip(),  # 좌우 반전
    transforms.RandomRotation(15),  # 최대 15도 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 밝기/대비 조정
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 테스트 데이터는 크기만 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
dataset_path = './dataset'  # 데이터셋 경로
dataset = NumericFolderDataset(root=dataset_path, transform=None)

# 데이터셋 분할 (Train/Test 8:2 비율)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 각각에 Transform 적용
train_dataset.dataset.transform = train_transforms
test_dataset.dataset.transform = test_transforms

# DataLoader 생성
batch_size = 32  # 배치 크기
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 확인
print("클래스 맵핑:", dataset.class_to_idx)
print(f"Train 데이터 크기: {len(train_dataset)}")
print(f"Test 데이터 크기: {len(test_dataset)}")


In [ ]:
# for debug

for x, y in train_loader:
  print(x.shape)
  print(y)
  break


torch.Size([32, 3, 224, 224])
tensor([ 2, 17,  9,  1,  0, 82, 34, 58, 33, 23, 55, 86, 12,  3, 87, 12, 77, 31,
        31, 23, 33,  1, 57, 21, 28, 29, 73, 67, 23, 67, 46,  2])


## **Model**

In [ ]:
%%writefile mymodel.py

from torchvision import models
import torch
import torch.nn as nn


# Model
class Custom_ResNet(nn.Module) :
  def __init__(self):
    super().__init__()

    self.resnet = models.resnet18(pretrained=True)

    for p in self.resnet.parameters():
      p.requires_grad = False

    # add layer
    self.resnet.fc = torch.nn.Identity()

    self.fc_mid = nn.Linear(512, 256)  # New fully connected layer

    self.resnet.fc_building = nn.Linear(256, 88)


  def forward(self, x):
    out = self.resnet(x)
    out = torch.relu(self.fc_mid(out))
    building_class = self.resnet.fc_building(out)

    return building_class

Overwriting mymodel.py


## **Train**

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import torch
from building_info import get_building_label
from mymodel import Custom_ResNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = Custom_ResNet()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
criterion = nn.CrossEntropyLoss()


# Epochs 설정
num_epochs = 35

num_classes = 88
best_acc = 0.0

for epoch in range(num_epochs):
  model.train()
  for x, y in train_loader:
    x, y = x.to(device), y.to(device)

    out = model(x)
    loss = criterion(out, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  sec1_correct = 0.0
  sec2_correct = 0.0
  val_loss = 0.0


  with torch.no_grad():
    for x, y in test_loader:
      x, y = x.to(device), y.to(device)

      out = model(x)
      out = F.softmax(out, dim=1)
      # print(out)

      # print(out.argmax(1).float(), y)


      # Section 1 - Predict appropriate subclass -> correct
      sec1_correct += (out.argmax(1) == y).float().sum().item()
      val_loss += criterion(out, y).item()

      # Section 2 - Add all probability of subclasses and Pick the best one for test
      batch, sub_class = out.shape

      # print(batch, sub_class)

      # 기존 매핑 (class_to_idx)
      class_to_idx = {
          '0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11,
          '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22,
          '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33,
          '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44,
          '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '58': 54, '59': 55,
          '6': 56, '60': 57, '61': 58, '62': 59, '63': 60, '64': 61, '65': 62, '66': 63, '67': 64, '68': 65, '69': 66,
          '7': 67, '70': 68, '71': 69, '72': 70, '73': 71, '74': 72, '75': 73, '76': 74, '77': 75, '78': 76, '79': 77,
          '8': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '9': 87
      }

      # 매핑을 뒤집기
      idx_to_class = {v: int(k) for k, v in class_to_idx.items()}

      # 예측된 값 -> 원래 클래스 이름으로 변환
      def get_original_class(predicted_label):
          """
          예측된 숫자를 원래 폴더 이름으로 변환
          :param predicted_label: 모델이 예측한 레이블 (int)
          :return: 원래 폴더 이름 (str)
          """
          return idx_to_class.get(predicted_label, "Unknown")  # 존재하지 않는 레이블이면 "Unknown" 반환


      new_out = torch.zeros(batch, 28)
      new_out = new_out.to(device)


      for i in range(batch):
        for j in range(num_classes):
          real_class = get_original_class(j)
          folder_label = get_building_label(real_classes)  # `j`를 폴더 이름으로 변환
          new_out[i][folder_label - 1] += out[i][j]


      # print(new_out)

      #print(y)
      for i in range(batch):
        y[i] = get_building_label(int(y[i]))

      #print(y)
      sec2_correct += (new_out.argmax(1) == y).float().sum().item()


  val_loss_avg = val_loss / len(test_loader)
  sec1_accuracy = 100. * sec1_correct / len(test_loader.dataset)
  sec2_accuracy = 100. * sec2_correct / len(test_loader.dataset)

  #### best.pth 저장하기
  if best_acc < sec1_accuracy:
    best_acc = sec1_accuracy
    torch.save(model.state_dict(), f'./best_weights.pth')
    print(f"Epoch {epoch + 1}: New best accuracy achieved! Model saved.")

  print(f"Epoch {epoch + 1}/{num_epochs} - "
    f"Validation Loss: {val_loss_avg:.4f}, "
    f"Section1 Accuracy: {sec1_accuracy:.2f}%",
    f"Section2 Accuracy: {sec2_accuracy:.2f}%")

  scheduler.step()

torch.save(model.state_dict(), f'./final_weights.pth')


Using device: cuda:0


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: New best accuracy achieved! Model saved.
Epoch 1/35 - Validation Loss: 4.4176, Section1 Accuracy: 14.93% Section2 Accuracy: 4.52%
Epoch 2: New best accuracy achieved! Model saved.
Epoch 2/35 - Validation Loss: 4.3587, Section1 Accuracy: 35.75% Section2 Accuracy: 5.88%
Epoch 3: New best accuracy achieved! Model saved.
Epoch 3/35 - Validation Loss: 4.2669, Section1 Accuracy: 53.85% Section2 Accuracy: 5.88%
Epoch 4: New best accuracy achieved! Model saved.
Epoch 4/35 - Validation Loss: 4.1610, Section1 Accuracy: 70.14% Section2 Accuracy: 0.45%


## **Test Code**

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import os

from building_info import get_building_label, num_classes, get_building_name
from mymodel import Custom_ResNet
from PIL import Image

# 기존 매핑 (class_to_idx)
class_to_idx = {
    '0': 0, '1': 1, '10': 2, '11': 3, '12': 4, '13': 5, '14': 6, '15': 7, '16': 8, '17': 9, '18': 10, '19': 11,
    '2': 12, '20': 13, '21': 14, '22': 15, '23': 16, '24': 17, '25': 18, '26': 19, '27': 20, '28': 21, '29': 22,
    '3': 23, '30': 24, '31': 25, '32': 26, '33': 27, '34': 28, '35': 29, '36': 30, '37': 31, '38': 32, '39': 33,
    '4': 34, '40': 35, '41': 36, '42': 37, '43': 38, '44': 39, '45': 40, '46': 41, '47': 42, '48': 43, '49': 44,
    '5': 45, '50': 46, '51': 47, '52': 48, '53': 49, '54': 50, '55': 51, '56': 52, '57': 53, '58': 54, '59': 55,
    '6': 56, '60': 57, '61': 58, '62': 59, '63': 60, '64': 61, '65': 62, '66': 63, '67': 64, '68': 65, '69': 66,
    '7': 67, '70': 68, '71': 69, '72': 70, '73': 71, '74': 72, '75': 73, '76': 74, '77': 75, '78': 76, '79': 77,
    '8': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '9': 87
}

# 매핑 뒤집기
idx_to_class = {v: int(k) for k, v in class_to_idx.items()}  # 폴더 이름을 숫자로 변환

# 예측된 값 -> 원래 클래스 이름(숫자)으로 변환
def get_original_class(predicted_label):
    """
    예측된 숫자를 원래 폴더 이름(숫자)으로 변환
    :param predicted_label: 모델이 예측한 레이블 (int)
    :return: 원래 폴더 이름 (int)
    """
    return idx_to_class.get(predicted_label, -1)  # 존재하지 않는 레이블이면 -1 반환


def single_img_test(img_path, weight_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Custom_ResNet()
    model.load_state_dict(torch.load(weight_path, map_location=device))
    model = model.to(device)
    model.eval()  # 평가 모드로 설정

    test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),  # 테스트 데이터는 크기만 조정
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_path = img_path  # 테스트 이미지 경로
    img = Image.open(image_path)
    img_tensor = test_transforms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(img_tensor)
        #print(out.shape)
        out = F.softmax(out, dim=1)

        # Section 1: 예측된 세부 클래스
        #print(out)
        #print(out.argmax(1))
        s1_class_idx = get_original_class(out.argmax(1).item())
        #print(f'{s1_class_idx} - {image_path}')
        s1_building_name = get_building_name(get_building_label(s1_class_idx))

        # Section 2: 새로운 클래스 구성 (건물별 확률 합산)
        batch, subclass = out.shape
        new_out = torch.zeros(batch, 28).to(device)

        for i in range(batch):
            for j in range(num_classes):
                folder_label = get_building_label(get_original_class(j))
                new_out[i][folder_label-1] += out[i][j]

        s2_class_idx = new_out.argmax(1).item()
        s2_building_name = get_building_name(s2_class_idx+1)

    print(f"Image Path: {image_path}",
          f"Section 1 - Predicted Detailed Class: {s1_building_name}",
          f"Section 2 - Predicted General Class: {s2_building_name}")
    #return s1_class_idx, s2_class_idx
    # s1_class_idx : section1 subclass 예측
    # s1_building_name : section1 building name output
    # s2_class_idx : section2 building class 합산 예측
    # s2_building_name : section2 building name output

img_dir = './testset'   # testdataset directory path
weight_path = './pth/8_weights.pth' #model weight path


image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')  # 원하는 확장자 추가 가능
image_paths = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(image_extensions)]

for image in image_paths:
    single_img_test(image, weight_path)